In [1]:
uid_score_bid = ['A,1,a','A,1,b','A,1,d','B,1,b','B,1,c','B,1,e','C,1,c','C,1,d','D,1,b','D,1,c','D,1,d','E,1,a','E,1,d']

In [2]:
#dict.setdefault : dict.setdefault(key, default=None)
#if key exist return the correspond value
#otherwise return default value
train = dict()
for rec in uid_score_bid:
    user,score,item = rec.split(",")
    train.setdefault(user,{})  
    train[user][item] = int(float(score))

In [3]:
train

{'A': {'a': 1, 'b': 1, 'd': 1},
 'B': {'b': 1, 'c': 1, 'e': 1},
 'C': {'c': 1, 'd': 1},
 'D': {'b': 1, 'c': 1, 'd': 1},
 'E': {'a': 1, 'd': 1}}

In [5]:
c = dict() #item matrix for cooccurance, if item a,b all in user mike's record
n = dict() #how many users buought this item
for user, record in train.items():
    for item in record.keys():
        n.setdefault(item, 0)
        n[item] += 1#find a record add 1
        c.setdefault(item,{})
        for other_item in record.keys():
            if other_item == item:
                continue
            c[item].setdefault(other_item, 0)
            c[item][other_item] += 1#item i j have both exist in this record

        

In [6]:
c

{'a': {'b': 1, 'd': 2},
 'b': {'a': 1, 'c': 2, 'd': 2, 'e': 1},
 'c': {'b': 2, 'd': 2, 'e': 1},
 'd': {'a': 2, 'b': 2, 'c': 2},
 'e': {'b': 1, 'c': 1}}

In [8]:
n

{'a': 2, 'b': 3, 'c': 3, 'd': 4, 'e': 1}

In [10]:
import math
#similarity metric: item i,j's coexist times/sqrt((total i time) * (total j time))
sim = dict()
for i,co_exist in c.items():
    sim.setdefault(i,{})
    for ij in co_exist.keys():
        sim[i][ij] = co_exist[ij] / math.sqrt(n[i] * n[ij])

In [11]:
sim

{'a': {'b': 0.4082482904638631, 'd': 0.7071067811865475},
 'b': {'a': 0.4082482904638631,
  'c': 0.6666666666666666,
  'd': 0.5773502691896258,
  'e': 0.5773502691896258},
 'c': {'b': 0.6666666666666666,
  'd': 0.5773502691896258,
  'e': 0.5773502691896258},
 'd': {'a': 0.7071067811865475,
  'b': 0.5773502691896258,
  'c': 0.5773502691896258},
 'e': {'b': 0.5773502691896258, 'c': 0.5773502691896258}}

In [16]:
#recommend strategy: based on histoic record for each item the user bought, for one item i
#select k item which is similiar with this item
#calculate k items recommendation score
#the score is score i from user record * similarity i,j( j from k items)
#an example for user A ,k = 3
k = 3
rank = dict()
user_record = train['A']
for item, score in user_record.items():#in this example the score 1 means the user buy this or click this item
    sorted_sim_score = sorted(sim[item].items(), key = lambda x:x[1], reverse = True)[:k]
    for other_item, sim_score in sorted_sim_score:
        if other_item in user_record.keys():
            continue
        rank.setdefault(other_item,0)
        rank[other_item] += sim_score * score
print sorted(rank.items(), key = lambda x:x[1], reverse = True)

[('c', 1.2440169358562925), ('e', 0.5773502691896258)]


In [23]:
#if a user he also is a retailer, which means he will but a lot items in this website and don't have preference
#this will impact our model performance
# add regularization: for each item have k items have relationship
#every time calculate similarity score divide the max score
c = dict() #item matrix for cooccurance, if item a,b all in user mike's record
n = dict() #how many users buought this item
for user, record in train.items():
    for item in record.keys():
        n.setdefault(item, 0)
        n[item] += 1#find a record add 1
        c.setdefault(item,{})
        for other_item in record.keys():
            if other_item == item:
                continue
            c[item].setdefault(other_item, 0)
            c[item][other_item] += 1/math.log(1+len(record)*1.0)# add penality for who buy many things
            
sim = dict()
sim_max = dict()
for i,co_exist in c.items():
    sim.setdefault(i,{})
    sim_max.setdefault(i,0.0)
    for ij in co_exist.keys():
        sim[i][ij] = co_exist[ij] / math.sqrt(n[i] * n[ij])
        if sim[i][ij] > sim_max[i]:
            sim_max[i] = sim[i][ij]
for item, sim_score_matrix in sim.items():
    for related_item in sim_score_matrix.keys():
        sim[item][related_item] = sim[item][related_item] / sim_max[item]

In [20]:
n

{'a': 2, 'b': 3, 'c': 3, 'd': 4, 'e': 1}

In [21]:
c

{'a': {'b': 0.7213475204444817, 'd': 1.631586747071319},
 'b': {'a': 0.7213475204444817,
  'c': 1.4426950408889634,
  'd': 1.4426950408889634,
  'e': 0.7213475204444817},
 'c': {'b': 1.4426950408889634,
  'd': 1.631586747071319,
  'e': 0.7213475204444817},
 'd': {'a': 1.631586747071319,
  'b': 1.4426950408889634,
  'c': 1.631586747071319},
 'e': {'b': 0.7213475204444817, 'c': 0.7213475204444817}}

In [24]:
sim

{'a': {'b': 0.5105093993383438, 'd': 1.0},
 'b': {'a': 0.6123724356957947,
  'c': 1.0,
  'd': 0.8660254037844387,
  'e': 0.8660254037844387},
 'c': {'b': 1.0, 'd': 0.9794138964885573, 'e': 0.8660254037844387},
 'd': {'a': 1.0, 'b': 0.721969316263228, 'c': 0.8164965809277261},
 'e': {'b': 1.0, 'c': 1.0}}